# 🚀 Customize and Deploy `openai/gpt-oss-120B` on Amazon SageMaker AI
---
In this notebook, we explore **GPT-OSS-120B**, a flagship open-weight reasoning model from OpenAI. You’ll learn how to fine-tune it on your data, evaluate its strong reasoning, tool-use, and code capabilities, and deploy it using SageMaker for high-performance use cases.

**What is GPT-OSS-120B?**

Released on **August 5, 2025** under the **Apache-2.0** license, GPT-OSS-120B is one of OpenAI’s new open-source models focused on reasoning, efficiency, and real-world usability. It is text-only (text-in / text-out) and designed for large context and complex tasks.  
🔗 Model card: [openai/gpt-oss-120b on Hugging Face](https://huggingface.co/openai/gpt-oss-120b)

---

**Key Specifications**

| Feature | Details |
|---|---|
| **Total Parameters** | ~117 billion |
| **Active Parameters per Token** | ~5.1 billion |
| **Model Type / Architecture** | Transformer with Mixture-of-Experts (MoE): 128 experts total, 4 active per token |
| **Number of Layers** | 36 |
| **Context Length** | Up to **128,000 tokens** |
| **Positional Embedding** | Rotary Positional Embedding (RoPE) |
| **Attention Patterns** | Alternating dense and locally banded sparse attention + grouped multi-query attention |
| **Modalities** | Text-only |
| **License** | Apache-2.0 |

---

**Benchmarks & Behavior**

- Matches or exceeds the performance of OpenAI’s **o4-mini** on core reasoning benchmarks.  
- Strong results on competition math (AIME), tool-use, code generation, and HealthBench tasks.  
- Optimized to run on a single 80 GB GPU (e.g., NVIDIA H100 or AMD MI300X), making it viable for production deployment at scale.  

---

**Using This Notebook**

Here’s what you’ll cover:

* Load a sample dataset from Hugging Face and prepare it for fine-tuning  
* Fine-tune with SageMaker Training Jobs  
* Run Model Evaluation  
* Deploy to SageMaker Endpoints  



In [1]:
%pip install -Uq sagemaker datasets

In [2]:
import boto3
import sagemaker
from PIL import Image
import torch

In [3]:
region = boto3.Session().region_name

from sagemaker.local import LocalSession 
sess = LocalSession() #sagemaker.Session(boto3.Session(region_name=region))
sess.config = {"local": {"local_code": True}}

sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

In [4]:
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

## Data Preparation for Supervised Fine-tuning

### [NuminaMath-CoT](https://huggingface.co/datasets/AI-MO/NuminaMath-CoT)

**NuminaMath-CoT** is a large-scale dataset of **~860,000+ math competition question-solution pairs**, designed to support chain-of-thought reasoning in mathematical problem solving.

**Data Format & Structure**:
- Each example is a question followed by a solution; the solution is formatted with detailed **Chain-of-Thought (CoT)** reasoning.  
- The data sources include *Chinese high school math exercises*, *US and international mathematics competition problems*, *online test-papers PDFs*, and *math discussion forums*.  
- Preprocessing includes OCR from PDFs, segmentation to extract problem-solution pairs, translation into English, alignment into CoT style, and formatting of final answers.  

**License**: Released under the **Apache-2.0** license.  

**Applications**:

This dataset is useful for training and evaluating models on tasks including:  
- Complex math problem solving with reasoning steps (algebra, geometry, number theory, etc.)  
- Benchmarking chain-of-thought performance of LLMs on competition-level math tasks  
- Educational tools and tutoring systems that require explainable math solutions  
- Fine-tuning models to improve consistency, reasoning depth, and accuracy in mathematical domains  


In [ ]:
import os
import json
import pprint
from tqdm import tqdm
from datasets import load_dataset

In [ ]:
dataset_parent_path = os.path.join(os.getcwd(), "tmp_cache_local_dataset")
os.makedirs(dataset_parent_path, exist_ok=True)

**Preparing Your Dataset in `messages` format**

This section walks you through creating a conversation-style dataset—the required `messages` format—for directly training LLMs using SageMaker AI.

**What Is the `messages` Format?**

The `messages` format structures instances as chat-like exchanges, wrapping each conversation turn into a role-labeled JSON array. It’s widely used by frameworks like TRL.

Example entry:

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "How do I bake sourdough?" },
    { "role": "assistant", "content": "First, you need to create a starter by..." }
  ]
}


In [ ]:
dataset_name = "Josephgflowers/Finance-Instruct-500k"
dataset = load_dataset(dataset_name, split="train[:1000]")

In [ ]:
pprint.pp(dataset[0])

In [ ]:
print(f"total number of fine-tunable samples: {len(dataset)}")

In [ ]:
def convert_to_messages_reasoning(row):
    system_content = "You are a mathematical reasoning assistant. Read the problem, restate the key givens and goal, then solve step-by-step with clear algebra (use LaTeX), keeping exact arithmetic (fractions/surds) and justifying each transformation (e.g., equal differences for arithmetic sequences). Verify any domain or extraneous-solution constraints, and present the final simplified answer concisely on the last line."
    
    user_content = row["problem"]
    
    assistant_content = row["answer"]

    reasoning_content = row["solution"]
    
    return {
        "messages": [
            { "role": "system", "content": system_content, "thinking": None},
            { "role": "user", "content": user_content, "thinking": None },
            { "role": "assistant", "content": assistant_content, "thinking": reasoning_content }
        ]
    }
    
    
dataset = dataset.map(convert_to_messages_reasoning, remove_columns=dataset.column_names)

In [ ]:
dataset_filename = os.path.join(dataset_parent_path, f"{dataset_name.replace('/', '--').replace('.', '-')}.jsonl")
dataset.to_json(dataset_filename, lines=True)

#### Upload file to S3

In [ ]:
from sagemaker.s3 import S3Uploader

In [ ]:
data_s3_uri = f"s3://{sess.default_bucket()}/dataset"

uploaded_s3_uri = S3Uploader.upload(
    local_path=dataset_filename,
    desired_s3_uri=data_s3_uri
)
print(f"Uploaded {dataset_filename} to > {uploaded_s3_uri}")

## Fine-Tune LLMs using SageMaker `Estimator`/`ModelTrainer`

In [ ]:
import time
from sagemaker.pytorch import PyTorch
from getpass import getpass
import yaml
from jinja2 import Template

In [ ]:
hf_token = getpass()

### Training using `PyTorch` Estimator

**Training Using `PyTorch` Estimator**
Leverages the official PyTorch SageMaker container to run a custom training script using the Accelerate and DeepSpeed libraries. This option is ideal for users who want full control over the training pipeline 

---
**Observability**: SageMaker AI has [SageMaker MLflow](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow.html) which enables you to accelerate generative AI by making it easier to track experiments and monitor performance of models and AI applications using a single tool.

You can choose to include MLflow as a part of your training workflow to track your model fine-tuning metrics in realtime by simply specifying a **mlflow** tracking arn.

Optionally you can also report to : **tensorboard**, **wandb**.

In [ ]:
MLFLOW_TRACKING_SERVER_ARN = None # or "arn:aws:sagemaker:us-west-2:<account-id>:mlflow-tracking-server/<server-name>"

if MLFLOW_TRACKING_SERVER_ARN:
    reports_to = "mlflow"
else:
    reports_to = "tensorboard"

In [ ]:
job_name = 'microsoft--Phi-3-mini-128k-instruct'
training_instance_type = "local_gpu"

In [ ]:
if MLFLOW_TRACKING_SERVER_ARN:
    training_env = {
        "MLFLOW_EXPERIMENT_NAME": f"exp-{job_name}",
        "MLFLOW_TAGS": '{"source.job": "sm-training-jobs", "source.type": "sft", "source.framework": "pytorch"}',
        "HF_TOKEN": hf_token,
        "MLFLOW_TRACKING_URI": MLFLOW_TRACKING_SERVER_ARN,
    }
else:
    training_env = {
        "HF_TOKEN": hf_token
    }

In [ ]:
pytorch_image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:2.8.0-gpu-py312-cu129-ubuntu22.04-sagemaker"
print(f"Using image: {pytorch_image_uri}")

In [ ]:
%%writefile sagemaker_code/requirements.text
torch==2.7.1
torchvision==0.22.1 
torchaudio==2.7.1
transformers==4.55.0
peft==0.17.0
accelerate==1.10.0
bitsandbytes==0.46.1
datasets==4.0.0
deepspeed==0.17.5
hf-transfer==0.1.8
hf_xet
liger-kernel==0.6.1
lm-eval[api]==0.4.9
kernels>=0.9.0
mlflow
Pillow
safetensors>=0.6.2
sagemaker==2.251.1
sagemaker-mlflow==0.1.0
sentencepiece==0.2.0
tokenizers>=0.21.4
triton
trl==0.21.0
tensorboard
psutil
py7zr
git+https://github.com/triton-lang/triton.git@main#subdirectory=python/triton_kernels
vllm==0.10.1
poetry
yq

#### Training strategy: `PeFT/LoRA`

In [ ]:
pytorch_estimator = PyTorch(
    image_uri=pytorch_image_uri,
    entry_point="sm_accelerate_train.sh", # Adapted bash script to train using accelerate on SageMaker - Multi-GPU
    source_dir="sagemaker_code",
    instance_type=training_instance_type,
    instance_count=1,
    base_job_name=f"{job_name}-pytorch",
    role=role,
    volume_size=300,
    py_version="py312",
    keep_alive_period_in_seconds=3600,
    environment=training_env,
    sagemaker_session=sess,
    hyperparameters={
        "config": "hf_recipes/openai/gpt-oss-120b--vanilla-peft-qlora.yaml"
    }
)

# fit or train
pytorch_estimator.fit(
    {"training": uploaded_s3_uri}, 
    wait=False
)

In [ ]:
s3_model_data_uri = pytorch_estimator.model_data
print(f"Fine-tuned model location: {s3_model_data_uri}")

#### Training strategy: `Spectrum`

In [ ]:
pytorch_estimator = PyTorch(
    image_uri=pytorch_image_uri,
    entry_point="sm_accelerate_train.sh", # Adapted bash script to train using accelerate on SageMaker - Multi-GPU
    source_dir="sagemaker_code",
    instance_type=training_instance_type,
    instance_count=1,
    base_job_name=f"{job_name}-pytorch",
    role=role,
    volume_size=300,
    py_version="py312",
    keep_alive_period_in_seconds=3600,
    environment=training_env,
    sagemaker_session=sess,
    hyperparameters={
        "config": "hf_recipes/openai/gpt-oss-120b--vanilla-spectrum.yaml"
    }
)

# fit or train
pytorch_estimator.fit(
    {"training": uploaded_s3_uri}, 
    wait=False
)

In [ ]:
s3_model_data_uri = pytorch_estimator.model_data
print(f"Fine-tuned model location: {s3_model_data_uri}")

#### Training strategy: `Full-Finetuning`

In [ ]:
pytorch_estimator = PyTorch(
    image_uri=pytorch_image_uri,
    entry_point="sm_accelerate_train.sh", # Adapted bash script to train using accelerate on SageMaker - Multi-GPU
    source_dir="sagemaker_code",
    instance_type=training_instance_type,
    instance_count=1,
    base_job_name=f"{job_name}-pytorch",
    role=role,
    volume_size=300,
    py_version="py312",
    keep_alive_period_in_seconds=3600,
    environment=training_env,
    sagemaker_session=sess,
    hyperparameters={
        "config": "hf_recipes/openai/gpt-oss-120b--vanilla-full.yaml"
    }
)

# fit or train
pytorch_estimator.fit(
    {"training": uploaded_s3_uri}, 
    wait=False
)

In [ ]:
s3_model_data_uri = pytorch_estimator.model_data
print(f"Fine-tuned model location: {s3_model_data_uri}")